In [1]:
from google.colab import drive
drive.mount('/content/drive') #replace with drive.mount('/content/drive/', force_remount=True) if the drive has changed since last mount in order to force the remount
%cd /content/drive/MyDrive/Neural-Highlighting-of-Affordance-Regions/

Mounted at /content/drive
/content/drive/MyDrive/Neural-Highlighting-of-Affordance-Regions


In [4]:
!apt-get update
!apt-get install -y xvfb ffmpeg libsm6 libxext6
!pip install open3d pyvirtualdisplay

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,197 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,632 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 4,225 kB in 2s (2,028 kB/s)
Reading pack

In [5]:
import os
import open3d as o3d
from pyvirtualdisplay import Display

name = 'dogPC_d_4_augs_1'
output_dir = './output_{}_{}_OK/'.format(name, 0)
log_dir = output_dir


display = Display(visible=0, size=(1400, 900))
display.start()
mesh_o3d = o3d.io.read_triangle_mesh(os.path.join(log_dir, f"{name}.ply"))

if not mesh_o3d.has_vertex_normals():
  mesh_o3d.compute_vertex_normals()

point_cloud = mesh_o3d.sample_points_poisson_disk(number_of_points=10000)

width_final_render, height_final_render = 1400, 900
render_final_pc = o3d.visualization.rendering.OffscreenRenderer(width_final_render, height_final_render)
material = o3d.visualization.rendering.MaterialRecord()
material.shader = "defaultUnlit"
render_final_pc.scene.add_geometry("point_cloud", point_cloud, material)

zoom_out_factor = 0.5
bounding_box = point_cloud.get_axis_aligned_bounding_box()
center = bounding_box.get_center()
extent = bounding_box.get_extent()
render_final_pc.scene.camera.look_at(center, center + [0, 0, 1], [0, 1, 0])
render_final_pc.scene.camera.set_projection(60 / zoom_out_factor, width_final_render / height_final_render, 0.1, 100.0,
                                      o3d.visualization.rendering.Camera.FovType.Horizontal)
pc_img = render_final_pc.render_to_image()
output_file = os.path.join(log_dir, f"{name}_final_render.jpg")
o3d.io.write_image(output_file, pc_img)
display.stop()

[Open3D INFO] EGL headless mode enabled.
